### Restaurant Order Analysis

Analyze order data to identify the most and least popular menu items and types of cuisine

#### Problem Statement:

You've just been hired as a Data Analyst for the Taste of the World Cafe, a restaurant that has diverse menu offerings and serves generous portions. The Taste of the World Cafe debuted a new menu at the start of the year. You've been asked to dig into the customer data to see which menu items are doing well / not well and what the top customers seem to like the most.

#### Objectives: 
<b>Objective 1: </b>Explore the items table

Your first objective is to better understand the items table by finding the number of rows in the table, the least and most expensive items, and the item prices within each category.

<b>Objective 2: </b>Explore the orders table

Your second objective is to better understand the orders table by finding the date range, the number of items within each order, and the orders with the highest number of items.

<b>Objective 3: </b>Analyze customer behavior

Your final objective is to combine the items and orders tables, find the least and most ordered categories, and dive into the details of the highest spend orders.

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import mysql.connector

In [ ]:
# conn = mysql.connector.connect(
#     host="localhost",
#     user="root",
#     password="password",
#     database="restaurant_db"
# )

# query1 = "SELECT * FROM menu_items"
# query2 = "SELECT * FROM order_details"
# menu_items = pd.read_sql(query1, conn)
# order_details = pd.read_sql(query2, conn)

# conn.close()

d:\anaconda\envs\Deep_Learning\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
d:\anaconda\envs\Deep_Learning\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [ ]:
# menu_items.to_csv("menu_items.csv", index=False)
# order_details.to_csv("order_details.csv", index=False)